# Introduction

The goal of this notebook is extract the maximal number of galician sentences to build a corpus of sentences that coulld be useful to train a spacy model or get insights on the galician language itself.

The notebook is based on the file created in the *wiki_DOWNLOAD* notebook where  article sentences are stored, _articles20221201_ and renders a first version of a corpuscle, _wiki_gal_corpus.pkl_,  and a file with the corpus sentences _wiki_sents_gal.txt_.  

## Libraries & functions

In [1]:
from Galutils import *

import py3langid as langid


In [2]:
def get_spacy_predict(sents,lang=None,pos_exclude=[]):#'PUNCT','SYM','X']):
   
    
    def model_predict(sent,lang=None):
        
        if not lang:
            return
        res=[]
        sentence=[[i.text,i.pos_,i.lemma_] for i in lang(sent) if alpha_text(i.text,patt='0-9,\.;:\?\¿\!\¡\-\<\>\(\)\[\]\{\}') ]
        return [tuple(item) for item in sentence if not item[1] in pos_exclude]

    res=[]
    all_bow={}
    all_df={}
    all_lemma={}
    all_propn=[]
    for sent in sents:
        keys=[]
        res.append([(token,pos,lemma) for token,pos,lemma in model_predict(sent,lang)])
        for token,pos,lemma in res[-1]:
            
            if pos=='PROPN':
                all_propn.append((token,lemma))
            elif (len(alpha_text(token))==len(token)) and (len(alpha_text(lemma))==len(lemma)):
                
                all_bow[token]=all_bow.get(token,0)+1
               
                all_lemma[lemma]=all_lemma.get(lemma,0)+1
                keys.append(lemma)
                
                    
        for key in set(keys):
            all_df[key]=all_df.get(key,0)+1
    return (res,all_bow,all_df,all_lemma,Counter(all_propn))
                           

## Loading data & Language model

In [3]:
path=Path.cwd()
path_model=path.parent/'Model'
path_data=path.parent/'dataset'

In [4]:
nlp=spacy.load(path_model/'gl_lg')          

/home/luis/.local/lib/python3.10/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'pt_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### load articles

In [5]:
articles=pickle_var(path_data/'articles20221201_gl.pkl')
articles=[document(*item) for item in articles]
len(articles),articles[0]

(154876,
 document(title='Alimento', category=['Alimentación', 'Artigos que toda Wikipedia debería ter (Ciencia)'], user=['Alfonso Márquez'], text=["O 'alimento' é a substancia normalmente comida ou bebida por seres vivos.", 'O termo alimento inclúe tamén bebidas líquidas.', 'A comida é a principal fonte de enerxía e nutrición dos animais, e é xeralmente de orixe animal ou vexetal.', 'Diversas disciplinas encárganse do estudo dos alimentos desde enfoques distintos: a bioloxía estuda os mecanismos de obtención, dixestión e refugallo dos alimentos por parte dos organismos, a ecoloxía estuda as cadeas alimentarias, a química dos alimentos a composición dos alimentos e o xeito no que se metabolizan dentro do organismo, e a tecnoloxía dos alimentos estuda a elaboración, produción e uso dos produtos destinados ao consumo humano.', 'Fontes de alimentos', 'A maioría dos alimentos ten a súa orixe nas plantas.', 'Algúns alimentos obtéñense directamente das plantas; mais mesmo os animais que se u

In [6]:
sentences=[]
for art in articles:
    sentences+=[item for item in art.text if len(alpha_text(item,' -').split())>3]
len(sentences)

2838254

### Language

Trying to exclude sentences that cannot be `gl`. It is supposed that if a sentence can be `gl` then the _gl_ language will be in the top-5 rank of `langid`

In [7]:
def thread(sents,limit=5):
    return [langid.rank(s)[:limit] for s in sents]

In [8]:
%%time

lang=[]
for chunk in vectorice(sentences,thread_function=thread,max_workers=100):
    lang+=chunk

CPU times: user 11.9 s, sys: 1.95 s, total: 13.8 s
Wall time: 22min 30s


In [9]:
lang=[transpose(item)[0] for item in lang]

In [10]:
Counter(lang).most_common(10)

[(('gl', 'es', 'an', 'pt', 'ca'), 330756),
 (('gl', 'es', 'an', 'ca', 'pt'), 144336),
 (('gl', 'pt', 'es', 'an', 'br'), 136552),
 (('gl', 'es', 'pt', 'an', 'ca'), 100171),
 (('gl', 'es', 'pt', 'an', 'eo'), 90135),
 (('gl', 'pt', 'es', 'an', 'eo'), 86732),
 (('gl', 'es', 'an', 'pt', 'eo'), 85681),
 (('gl', 'es', 'pt', 'an', 'br'), 84215),
 (('gl', 'es', 'an', 'pt', 'oc'), 74966),
 (('gl', 'es', 'an', 'pt', 'br'), 71596)]

In [11]:
sents=list(set([s for s,l in zip(sentences,lang) if 'gl' in l]))
len(sents)

2613369

## Computing model

In [12]:
def thread(sents):
    return get_spacy_predict(sents,nlp)


In [13]:
%%time
answer=vectorice(sents,thread_function=thread,max_workers=100)

/home/luis/.local/lib/python3.10/site-packages/spacy/pipeline/attributeruler.py:150: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
/home/luis/.local/lib/python3.10/site-packages/spacy/pipeline/attributeruler.py:150: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
/home/luis/.local/lib/python3.10/site-packages/spacy/pipeline/attributeruler.py:150: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
/home/luis/.local/lib/python3.10/site-packages/spacy/pipeline/attributeruler.py:150: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
/home/luis/.local/lib/python3.10/site-packages/spacy/pipeline/attributeruler

CPU times: user 59.7 s, sys: 9.44 s, total: 1min 9s
Wall time: 45min 3s


In [14]:
%%time
all_toks=[]

for l in answer:
    toks,bow,df,lemma,propn=l
    all_toks+=toks


CPU times: user 115 ms, sys: 4.62 ms, total: 120 ms
Wall time: 119 ms


In [15]:
len(sents),len(all_toks)

(2613369, 2613369)

Filter sentences with at least one `VERB` or `AUX`

In [16]:
to_drop=[i for i,val in enumerate(all_toks) if not ('VERB' in (test:=transpose(val)[1]) or 'AUX' in test)]
len(to_drop)
         

332104

In [17]:
for indx in reversed(to_drop):
    all_toks.pop(indx)
    sents.pop(indx)

In [18]:
len(sents),len(all_toks)

(2281265, 2281265)

In [19]:
%%time

all_bow={}
all_df={}
all_lemma={}
all_propn={}
for l in answer:
    toks,bow,df,lemma,propn=l
   
    all_bow=update_dict(all_bow,bow)
    all_df=update_dict(all_df,df)
    all_lemma=update_dict(all_lemma,lemma)
    all_propn=update_dict(all_propn,propn)

CPU times: user 5.65 s, sys: 12.7 ms, total: 5.66 s
Wall time: 5.66 s


In [29]:
#Save vars
pickle_var(path_data/'wiki_gal_tokens.pkl',all_toks)
pickle_var(path_data/'wiki_gal_sents.pkl',sents)

In [34]:
(path_data/'wiki_sents_gal.txt').write_text('\n'.join(sents),encoding='utf8')

314669581

In [21]:
print(f'Num unic tokens : {len(all_bow)},\nNum unic lemmas: {len(all_df)}\nNum unic PROPN: {len(all_propn)}\nNum sentences: {len(all_toks)}')

Num unic tokens : 403153,
Num unic lemmas: 334994
Num unic PROPN: 521912
Num sentences: 2281265


In [22]:
Counter(all_df).most_common(50)

[('o', 1835198),
 ('de', 1661532),
 ('e', 1150597),
 ('ser', 907344),
 ('do', 854009),
 ('no', 782473),
 ('en', 774795),
 ('que', 740078),
 ('un', 715235),
 ('da', 620231),
 ('seu', 465242),
 ('como', 335481),
 ('a', 309189),
 ('por', 293497),
 ('para', 278623),
 ('ao', 271791),
 ('con', 227261),
 ('se', 223035),
 ('máis', 184982),
 ('á', 184308),
 ('polo', 165752),
 ('ou', 164424),
 ('non', 159420),
 ('ter', 156368),
 ('ano', 150407),
 ('estar', 145754),
 ('dos', 144794),
 ('este', 137845),
 ('co', 132778),
 ('tamén', 126096),
 ('primeiro', 125673),
 ('entre', 119530),
 ('poder', 117811),
 ('dun', 116063),
 ('outro', 96706),
 ('todo', 88708),
 ('dous', 87582),
 ('nun', 87062),
 ('parte', 79853),
 ('segundo', 78316),
 ('lugar', 77528),
 ('coa', 75832),
 ('durante', 74901),
 ('pola', 73567),
 ('sobre', 71483),
 ('pero', 69178),
 ('mesmo', 67501),
 ('concello', 64646),
 ('cidade', 63407),
 ('despois', 62740)]

#### Filter bow, df and lemma. Compute tfidf

In [23]:
all_bow={key:val for key,val in all_bow.items() if val >3}
all_lemma={key:val for key,val in all_lemma.items() if val >3}

all_df={key:val for key,val in all_df.items() if val >3}
inter_key=set(all_lemma.keys()).intersection(all_df.keys())

all_lemma={key:all_lemma[key] for key in inter_key}
all_df={key:all_df[key] for key in inter_key}

print(f'#BOW: {len(all_bow)}\n #Lemma: {len(all_lemma)},\n #DF: {len(all_df)}')

total_toks=sum(list(all_lemma.values()))
total_sents=len(all_toks)
all_idf={key:np.log(total_sents/val) for key,val in all_df.items()}
tfidf={key:val*all_lemma[key]/total_toks for key,val in all_idf.items()}

#BOW: 138432
 #Lemma: 102239,
 #DF: 102239


In [24]:
Counter(all_bow).most_common(20)

[('de', 3177387),
 ('e', 1586355),
 ('a', 1509639),
 ('o', 1184919),
 ('que', 1013607),
 ('en', 876364),
 ('do', 849747),
 ('da', 789768),
 ('un', 442180),
 ('no', 406984),
 ('na', 373931),
 ('unha', 370061),
 ('os', 367573),
 ('como', 351557),
 ('é', 346583),
 ('O', 308788),
 ('por', 308502),
 ('A', 304339),
 ('para', 299652),
 ('dos', 280278)]

In [25]:
Counter(all_lemma).most_common(20)

[('o', 3745101),
 ('de', 3192084),
 ('e', 1590137),
 ('do', 1169643),
 ('ser', 1054910),
 ('que', 1013613),
 ('no', 1004151),
 ('en', 1003520),
 ('un', 876077),
 ('da', 774344),
 ('seu', 564591),
 ('como', 370349),
 ('a', 346536),
 ('por', 328781),
 ('ao', 316089),
 ('para', 314509),
 ('se', 252431),
 ('con', 250781),
 ('máis', 203584),
 ('á', 201191)]

In [26]:
Counter(all_idf).most_common(20)

[('caroliniana', 13.253946310513896),
 ('colofonia', 13.253946310513896),
 ('fir', 13.253946310513896),
 ('drogaría', 13.253946310513896),
 ('incluínte', 13.253946310513896),
 ('influíren', 13.253946310513896),
 ('Cápsula', 13.253946310513896),
 ('burlor', 13.253946310513896),
 ('pincha', 13.253946310513896),
 ('practicabl', 13.253946310513896),
 ('contei', 13.253946310513896),
 ('carajo', 13.253946310513896),
 ('brazada', 13.253946310513896),
 ('apaixonada', 13.253946310513896),
 ('existencialisto', 13.253946310513896),
 ('inmunomodulatorio', 13.253946310513896),
 ('Contentos', 13.253946310513896),
 ('sosa', 13.253946310513896),
 ('utilizábao', 13.253946310513896),
 ('adelantar', 13.253946310513896)]

In [27]:
Counter(tfidf).most_common(20)

[('do', 0.025297675104996316),
 ('que', 0.02511780960374905),
 ('e', 0.023958033875053),
 ('en', 0.023855017408974794),
 ('no', 0.023652049138084077),
 ('un', 0.022368074784096345),
 ('de', 0.0222738536694409),
 ('da', 0.022199927303366587),
 ('ser', 0.021409408960429904),
 ('seu', 0.01975999140933233),
 ('o', 0.017937148966350448),
 ('como', 0.015627539582477652),
 ('a', 0.015245266942272278),
 ('por', 0.01484112759296261),
 ('ao', 0.014802824166388168),
 ('para', 0.014556953320788038),
 ('se', 0.012920232776729795),
 ('con', 0.012732160307103776),
 ('máis', 0.011258432446470889),
 ('ou', 0.01114335882840037)]

## Corpuscle object

In [28]:
corpus=corpuscle(nlp,all_bow,all_lemma,tfidf,all_propn)

pickle_var(path_model/'wiki_gal_corpuscle.pkl',corpus)